<a href="https://colab.research.google.com/github/pin164/Automated-Expense-Categorization/blob/main/Copy_of_Project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1
# Import  dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback


from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


## Preprocessing

In [2]:
# STEP 2
#  Import and read the Cleaned_Invoice_3.csv

application_df = pd.read_csv("/content/drive/MyDrive/Project 4/Cleaned_Invoice_3.csv",encoding='latin1').iloc[:,:14]
application_df.head()

,Column2,Month,Year,PO#,Invoice Date,Invoice#,total Invoice Amt,TRS,Project2,Amount,Cost Centre,Order#,Column3,Column1
0,vendor 1042,Mar,2022.0,6101206151,2022-03-31,275769,"$22,844.75",SDS9089963,Student Records Management System - Project-228,"$4,555.25",140548,311107202.0,ZDT,1
1,vendor 1042,Mar,2022.0,6101206151,2022-03-31,275769,"$22,844.75",SDS9089877,Travel MB GP Upgrade to 2016 - 0,$650.75,165039,311107504.0,FVN,2
2,vendor 1042,Mar,2022.0,6101206151,2022-03-31,275769,"$22,844.75",SDS9084483,Portable Benefit Form - XY11351,$274.00,205697,311107169.0,TT,3
3,vendor 1042,Mar,2022.0,6101206151,2022-03-31,275769,"$22,844.75",SDS9089761,MI - HIS Replacement Project (Project-027),"$9,076.25",100013025,311107192.0,MV,4
4,vendor 1042,Mar,2022.0,6101206151,2022-03-31,275769,"$22,844.75",SDS9090105,Corporater - Balanced Scorecard,"$4,521.00",195403; IO 100013486; PO 4501231722,311107217.0,SKJ,5


In [3]:
# STEP 3
# #extract beneficial columns only
X_df = application_df[['Column2','PO#','TRS','Order#','Column3']]
Y_df = application_df['Cost Centre']
X_df.head()


,Column2,PO#,TRS,Order#,Column3
0,vendor 1042,6101206151,SDS9089963,311107202.0,ZDT
1,vendor 1042,6101206151,SDS9089877,311107504.0,FVN
2,vendor 1042,6101206151,SDS9084483,311107169.0,TT
3,vendor 1042,6101206151,SDS9089761,311107192.0,MV
4,vendor 1042,6101206151,SDS9090105,311107217.0,SKJ


In [16]:
Y_df.value_counts()


,count
Cost Centre,
171318,1593
171293,869
171225,546
0,299
165155,195
...,...
195496,3
165212,2
930076150,1


In [4]:
X_df.dtypes

,0
Column2,object
PO#,object
TRS,object
Order#,float64
Column3,object


In [5]:
X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')

X_df.dtypes

<ipython-input-5-b5400605a5fc>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[311107202.0 311107504.0 311107169.0 ... nan nan 311107183.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')


,0
Column2,object
PO#,object
TRS,object
Order#,object
Column3,object


In [6]:
Y_df.head()

,Cost Centre
0,140548
1,165039
2,205697
3,100013025
4,195403; IO 100013486; PO 4501231722


In [7]:
application_df_encoded = pd.get_dummies(X_df)
application_df_encoded

,Column2_vendor 1001,Column2_vendor 1002,Column2_vendor 1003,Column2_vendor 1004,Column2_vendor 1005,Column2_vendor 1006,Column2_vendor 1007,Column2_vendor 1008,Column2_vendor 1009,Column2_vendor 1010,...,Column3_TT,Column3_TTB,Column3_TZN,Column3_TZN - LVR,Column3_VNR,Column3_VT,Column3_ZDJ,Column3_ZDK,Column3_ZDT,Column3_ZNT
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10561,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10562,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10563,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10564,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
# Display the first few rows of the newly encoded dataframe
# This will show how the categorical columns have been transformed into numerical ones
print("\nEncoded DataFrame (First 5 Rows):")
print(application_df_encoded.head())


Encoded DataFrame (First 5 Rows):
   Column2_vendor 1001  Column2_vendor 1002  Column2_vendor 1003  \
0                False                False                False   
1                False                False                False   
2                False                False                False   
3                False                False                False   
4                False                False                False   

   Column2_vendor 1004  Column2_vendor 1005  Column2_vendor 1006  \
0                False                False                False   
1                False                False                False   
2                False                False                False   
3                False                False                False   
4                False                False                False   

   Column2_vendor 1007  Column2_vendor 1008  Column2_vendor 1009  \
0                False                False                False   
1          

In [9]:
# Step 16: Split the Data into Features (X) and Target (y)

# The target variable is assumed. 'Cost Centre' is the target column.
#
y = Y_df.values  # Target variable

# The features will include all other columns except for the target
X = application_df_encoded.values  # Feature variables

# Split the Data into Training and Testing Sets

# Use train_test_split to split the data into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing, and 80% for training
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets to verify the split
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)


Training Features Shape: (8452, 1221)
Testing Features Shape: (2114, 1221)
Training Target Shape: (8452,)
Testing Target Shape: (2114,)


In [10]:
#Step 17:   Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [11]:
# Step 18: Create the Neural Network Model Using TensorFlow and Keras

# Step 18.1: Determine the Number of Input Features
# The number of input features is equal to the number of columns in X_train_scaled
num_features = X_train_scaled.shape[1]
print(f"Number of input features: {num_features}")



Number of input features: 1221


In [12]:

# Step 18.2: Initialize the Sequential Model
# The Sequential API buildsthe model layer by layer
nn = tf.keras.models.Sequential()

# Step 18.3: Add the First Hidden Layer
# - Number of neurons in the first layer
first_neuron = 80
# - Activation function: ReLU (Rectified Linear Unit) is commonly used for hidden layers
# - Input dim: Specifies the shape of the input data (only needed for the first layer)
nn.add(layers.Dense(first_neuron, activation='relu', input_dim=num_features))

# - `Dense` layer is a fully connected layer.
# - `first_neuron` neurons in this layer.
# - `activation='relu'` introduces non-linearity, allowing the model to learn complex patterns.
# - `input_dime=(num_features,)` specifies the shape of the input data.


# Step 18.4: Add the Second Hidden Layer
# Adding another hidden layer can help the model learn more complex patterns
# - Number of neurons in the second layer
second_neuron = 30
# - Activation function: ReLU
nn.add(layers.Dense(second_neuron, activation='relu'))
# - `second_neuron` neurons in the second hidden layer.
# - Using ReLU activation function.
# - No need to specify `input_shape` for subsequent layers as Keras automatically infers it.




# Step 18.5: Add the Output Layer
# - For binary classification, we use 1 neuron with a sigmoid activation function
# - Activation function: Sigmoid to output a probability between 0 and 1
nn.add(layers.Dense(1, activation='sigmoid'))

# - `1` neuron because we're predicting a single binary outcome.
# - `activation='sigmoid'` outputs a value between 0 and 1, representing the probability of the positive class.

# Step 18.6: Review the Model Structure
# This will print a summary of the model architecture, including layers, output shapes, and the number of parameters
# Check the structure of the model
nn.summary()
# - `model.summary()` provides a detailed overview of the model, which is useful for verifying the architecture.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │          97,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 100,221 (391.49 KB)

 Trainable params: 100,221 (391.49 KB)

 Non-trainable params: 0 (0.00 B)

Explanation of the Summary:

*   Layer Names: Each layer is named automatically (e.g., dense, dense_1, dense_2)
*   Output Shapes: Shows the shape of the output from each layer..
*   Param #: Indicates the number of parameters (weights and biases) in each layer.
*   Total Params: Total number of trainable parameters in the model.



In [13]:
# Step 19.1: Compile the Neural Network Model

# Compile the model with the following configurations:
# - Optimizer: Adam optimizer is efficient and widely used for training deep learning models.
# - Loss Function: Binary Crossentropy is suitable for binary classification tasks.
# - Metrics: Monitor 'accuracy' to evaluate the performance of the model.

nn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

# Explanation:
# - `optimizer='adam'`: Adam optimizer adjusts the learning rate during training for efficient convergence.
# - `loss='binary_crossentropy'`: Measures the performance of the model where the output is a probability value between 0 and 1.
# - `metrics=['accuracy']`: Tracks the accuracy of the model during training and evaluation.


In [14]:
# Step 19.2: Create a  Callback to Save Weights Every Five Epochs
class SaveEveryNEpochs(Callback):
    def __init__(self, save_freq, filepath):
        super(SaveEveryNEpochs, self).__init__()
        self.save_freq = save_freq
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs=None):
        # Save the model every 'save_freq' epochs
        if (epoch + 1) % self.save_freq == 0:
            self.model.save_weights(self.filepath.format(epoch=epoch + 1))
            print(f'\nSaved weights at epoch {epoch + 1}')

# Initialize the custom callback to save weights every five epochs
checkpoint_callback = SaveEveryNEpochs(
    save_freq=5,  # Save every 5 epochs
    filepath='weights_epoch_{epoch:02d}.weights.h5'  # Filepath must end with '.weights.h5'
)


In [15]:
# Step 19.3:

epochs = 50
batch_size = 10
# Train the model using the training data
# - X_train_scaled: Preprocessed and scaled training features
# - y_train: Training target variable
# - epochs: Number of times the entire training dataset is passed through the model
# - batch_size: Number of samples per gradient update
# - validation_data: Tuple (X_test_scaled, y_test) to evaluate the loss and any model metrics at the end of each epoch
# - callbacks: List of callback instances to apply during training (e.g., checkpoint_callback)

history = nn.fit(X_train_scaled, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(X_test_scaled, y_test),
                callbacks=[checkpoint_callback],
                verbose=1)  # Verbosity mode, 1 = progress bar

# Explanation:
# - `epochs=100`: The model will iterate 50 times over the entire training dataset.
# - `batch_size=50`: The model updates its weights after every 32 samples.
# - `validation_data=(X_test_scaled, y_test)`: The model will evaluate its performance on the testing data at the end of each epoch.
# - `callbacks=[checkpoint_callback]`: Incorporates the custom callback to save model weights every five epochs.
# - `verbose=1`: Displays a progress bar and training metrics during training.

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
# Step 19.4: Evaluate the Neural Network Model on Test Data

# Evaluate the trained model using the testing data
# - X_test_scaled: Preprocessed and scaled testing features
# - y_test: Testing target variable

test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=0)

# Print the evaluation results
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Explanation:
# - `nn.evaluate()`: Computes the loss and specified metrics for the model on the provided data.
# - `verbose=0`: Suppresses the progress bar during evaluation.
# - `test_loss`: The loss value on the testing data.
# - `test_accuracy`: The accuracy of the model on the testing data.



Test Loss: 0.5632
Test Accuracy: 0.7242


In [ ]:
# Step 19.5: Save the Trained Model to an HDF5 File

# Define the filename for the saved model
model_filename = 'AlphabetSoupCharity.h5'

# Save the entire model to the HDF5 file
nn.save(model_filename)

# Confirmation message
print(f"\nModel has been saved to {model_filename}")





Model has been saved to AlphabetSoupCharity.h5
